In [1]:
import earthaccess
import sys
import os
if r'C:\Users\attic\HLS Kelp Detection\tools' not in sys.path:
    sys.path.append(r'C:\Users\attic\HLS Kelp Detection\tools')
import kelp_tools_windows as kt
import earth_access_tools as eat
import mgrs
from pyproj import Proj, transform,Transformer
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import requests
import xml.etree.ElementTree as ET
import csv

In [8]:
def search_bounding_box(tile_id, csv_file=r'C:\Users\attic\HLS Kelp Detection\maps\tiling_boxes.csv', clip=False):
        with open(csv_file, mode='r') as file:
            csv_reader = csv.reader(file)
            coords = None
            for row in csv_reader:
                if row[0] == tile_id:
                    coords = row[1:]
                    break
            if coords is None:
                return None
        bounding_box=(float(coords[9]), float(coords[10]), float(coords[3]),float(coords[4]))
        print(coords)
        return bounding_box

In [2]:
eat.create_tiling_reference()

'C:\\Users\\attic\\HLS Kelp Detection\\maps\\tiling_boxes.csv'

In [9]:
search_bounding_box('11SKU')


['-120.2951835442', '35.1982077222', '0.0', '-119.0901471721', '35.2250276976', '0.0', '-119.0653735271', '34.235521621', '0.0', '-120.2561776636', '34.2096670331', '0.0']


(-120.2561776636, 34.2096670331, -119.0901471721, 35.2250276976)

In [4]:
hls_path = r'C:\Users\attic\HLS Kelp Detection\imagery\Isla_Vista_Kelp\11SKU\HLS.L30.T11SKU.2018186T183312.v2.0\HLS.L30.T11SKU.2018186T183312.v2.0.B02.tif'
tile = '11SKU'
dem_path = rf'C:\Users\attic\HLS Kelp Detection\imagery\Isla_Vista_Kelp\{tile}\dem'

In [5]:
eat.download_dem(tile_id=tile, dem_path=dem_path)

Granules found: 4
 Getting 1 granules, approx download size: 0.02 GB


QUEUEING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

File ASTGTMV003_N35W121_dem.tif already downloaded
File ASTGTMV003_N35W121_num.tif already downloaded


PROCESSING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2 [00:00<?, ?it/s]

...

 Getting 1 granules, approx download size: 0.01 GB


QUEUEING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

File ASTGTMV003_N34W121_dem.tif already downloaded
File ASTGTMV003_N34W121_num.tif already downloaded


PROCESSING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def reproject_dem_to_hls(hls_path, dem_path):
    with rasterio.open(hls_path) as dst:
        hls = dst.read()
        dem = rasterio.open(dem_path)
        if dem.crs != dst.crs:
            reprojected_dem = np.zeros((hls.shape[1], hls.shape[2]), dtype=hls.dtype)
            reproject(
                source=dem.read(),
                destination=reprojected_dem,
                src_transform=dem.transform,
                src_crs=dem.crs,
                dst_transform=dst.transform,
                dst_crs=dst.crs,
                resampling=Resampling.bilinear)
            if reprojected_dem.any():
                return reprojected_dem
            else:
                return False